In [1]:
import sympy as sp

In [2]:
import matplotlib.pyplot as plt

In [3]:
import sympy.physics.units.quantities as sq

In [4]:
from sympy.physics.quantum.constants import hbar

In [5]:
PSI_FUNCTION = sp.Function( "psi" )
POTENTIAL_FUNCTION = sp.Function( "V" )
TOTAL_ENERGY_SYMBOL = sq.Symbol( 'E', nonzero = True, positive = True )
MASS_SYMBOL = sq.Quantity( 'm', positive = True, nonzero = True )
#REDUCED_PLANCK_CONSTANT_SYMBOL = sq.Symbol( "hbar" )
POSITION_SYMBOL = sp.Symbol( 'x', positive = True )

def time_independent_schroedinger_equation( 
            psi = PSI_FUNCTION, 
            potential = POTENTIAL_FUNCTION, 
            total_energy = TOTAL_ENERGY_SYMBOL, 
            mass = MASS_SYMBOL, 
            reduced_planck_constant = hbar, #REDUCED_PLANCK_CONSTANT_SYMBOL, 
            position = POSITION_SYMBOL 
        ): 
    return sp.Eq( 
            ( ( -( reduced_planck_constant ** 2 ) / ( 2 * mass ) ) \
                    * sp.Derivative( psi( position ), ( position, 2 ) ) )
                    + ( potential( position ) * psi( position ) ), 
            total_energy * psi( position ) 
        )


In [6]:
set_equal = lambda to_set, value : sp.Eq( to_set, value )
both_sides = lambda equation, opreation : sp.Eq( opreation( equation.lhs ), opreation( equation.rhs ) )
equation_to_dict = lambda equation : { equation.lhs : equation.rhs }

In [7]:
hbar_eq = sp.Eq( hbar, sp.physics.units.planck / ( 2 * sp.pi ) )

In [8]:
hbar_eq

Eq(hbar, planck/(2*pi))

In [9]:
psi = PSI_FUNCTION
potential = POTENTIAL_FUNCTION
total_energy = TOTAL_ENERGY_SYMBOL
mass = MASS_SYMBOL
#reduced_planck_constant = REDUCED_PLANCK_CONSTANT_SYMBOL
x = POSITION_SYMBOL

In [10]:
k = sp.Symbol( 'k' )

In [11]:
k_squared = sp.Eq( k ** 2, ( ( 2 * total_energy * mass ) / ( hbar ** 2 ) ) )

In [12]:
k_squared

Eq(k**2, 2*m*E/hbar**2)

In [13]:
class ZeroPotential( sp.Function ): 
    @classmethod
    def eval( cls, position ):
        return 0

In [14]:
#https://docs.sympy.org/latest/modules/assumptions/index.html#querying
#https://docs.sympy.org/latest/modules/codegen.html#module-sympy.codegen.cxxnodes

class Potential( sp.Function ): 
    DEFAULT_POTENTIAL = sp.Symbol( "V_0" )
    @classmethod
    def eval( cls, position, potential = DEFAULT_POTENTIAL ): 
        return potential

In [15]:
class TunnelPotential( sp.Function ): 
    DEFAULT_WELL_LENGTH = sp.Symbol( 'L' )
    DEFAULT_POTENTIAL = Potential.DEFAULT_POTENTIAL
    DEFAULT_START = 0
    @classmethod
    def eval( cls, position, length = DEFAULT_WELL_LENGTH, 
             start = DEFAULT_START, potential = DEFAULT_POTENTIAL ): 
        if position < start or position > sp.simplify( length + start ): 
            return ZeroPotential.eval( position )
        return Potential.eval( position, potential )

In [16]:
class Stepper: 

    LEFT = "LEFT", 
    RIGHT = "RIGHT"
    
    def __init__( self, first_step, new_steps = None ): 
        self.steps = new_steps if new_steps else []
        self.steps.append( first_step )
        self.chaining = False
    
    def step_number( self, step = None ): 
        return ( len( self.steps ) - 1 ) if not step else step
    
    def last_step( self, step = None ):
        return self.steps[ self.step_number( step ) ]
    
    def _return_chain( self, step, chain ): 
        return self if ( chain or self.chaining ) else step        
    
    def add_step( self, new_step, chain = False ):
        self.steps.append( new_step )
        return self._return_chain( self.steps[ -1 ], chain )

    def operate( self, operation, step = None, chain = False ):
        self.steps.append( operation( self.last_step( step ) ) )
        return self._return_chain( self.steps[ -1 ], chain )

    def manipulate( self, operation, chain = False ): 
        return self._return_chain( self.add_step( 
                both_sides( self.last_step(), operation ) ), 
                chain 
            )
    
    def delete_step( self, step ): 
        old_step = self.steps[ step ]
        del self.steps[ step ]
        return old_step
    
    def undo( self, step = None, chain = False ): 
        return self._return_chain( 
                self.delete_step( self.step_number( step ) ), 
                chain 
            )
    
    def clone( self, from_step = None ):
        return self.branch( lambda blank : blank, from_step )
    
    def branch( self, operation = None, from_step = None ): 
        from_step = self.step_number( from_step )
        return Stepper( 
                both_sides( self.last_step( from_step ), operation ), 
                self.steps[ : self.step_number( from_step ) : ], 
            )
    
    def substitute_constant( self, constant, chain = False ):
            return self.operate( lambda step : step.subs( { constant.rhs : constant.lhs } ) )
    
    def negate_add( self, side_to_negate_add, chain = False ): 
        last_step = self.last_step()
        return self.manipulate( 
                lambda side : side + -( 
                    last_step.lhs if side_to_negate_add == Stepper.LEFT 
                            else last_step.rhs 
                        ), 
                chain 
            )
    
    def rename( self, old_symbol, new_symbol_name, chain = False ): 
        last_step = self.last_step()
        assert old_symbol in last_step.atoms() or old_symbol in last_step.atoms( sp.Function )
        return self.add_step( last_step.subs( 
            old_symbol if type( old_symbol ) == dict \
                    else { old_symbol : new_symbol_name } ), chain 
                )
    
    def chain( self, operation ):
        self.chaining = True
        operation( self )
        self.chaining = False
        return self
    
    def begin_chain( self ): 
        self.chaining = True
        return self
    
    def end_chain( self ): 
        self.chaining = False
        return self


In [17]:
k_squared

Eq(k**2, 2*m*E/hbar**2)

In [18]:
psi_region = [
        Stepper( time_independent_schroedinger_equation( potential = ZeroPotential ) ), 
        Stepper( time_independent_schroedinger_equation( potential = Potential ) ), 
        Stepper( time_independent_schroedinger_equation( potential = ZeroPotential ) )
    ]

In [19]:
psi_region[ 0 ].last_step()

Eq(-hbar**2*Derivative(psi(x), (x, 2))/(2*m), E*psi(x))

In [20]:
psi_region[ 0 ].manipulate( lambda side : ( side - psi_region[ 0 ].last_step().rhs ).simplify() )

Eq(-E*psi(x) - hbar**2*Derivative(psi(x), (x, 2))/(2*m), 0)

In [21]:
psi_region[ 0 ].manipulate( lambda side : -( side / total_energy ).simplify() )

Eq(psi(x) + hbar**2*Derivative(psi(x), (x, 2))/(2*m*E), 0)

In [22]:
psi_region[ 0 ].manipulate( lambda side : side - psi( x ) )

Eq(hbar**2*Derivative(psi(x), (x, 2))/(2*m*E), -psi(x))

In [23]:
psi_region[ 0 ].manipulate( lambda side : side * k_squared.rhs )

Eq(Derivative(psi(x), (x, 2)), -2*m*E*psi(x)/hbar**2)

In [24]:
psi_region[ 0 ].manipulate( lambda side : side / psi( x ) )

Eq(Derivative(psi(x), (x, 2))/psi(x), -2*m*E/hbar**2)

In [25]:
k_squared

Eq(k**2, 2*m*E/hbar**2)

In [26]:
psi_region[ 0 ].substitute_constant( k_squared )

Eq(Derivative(psi(x), (x, 2))/psi(x), -k**2)

In [27]:
psi_region[ 0 ].manipulate( lambda side : side * psi( x ) )

Eq(Derivative(psi(x), (x, 2)), -k**2*psi(x))

In [28]:
psi_region[ 0 ].negate_add( Stepper.RIGHT )

Eq(k**2*psi(x) + Derivative(psi(x), (x, 2)), 0)

In [29]:
# For later

psi_region[ 2 ] = psi_region[ 0 ].clone()

In [30]:
psi_region[ 2 ].last_step()

Eq(k**2*psi(x) + Derivative(psi(x), (x, 2)), 0)

In [31]:
k_0 = sp.Symbol( "k_0" )

In [32]:
psi_region[ 0 ].rename( k, k_0 )

Eq(k_0**2*psi(x) + Derivative(psi(x), (x, 2)), 0)

In [33]:
diff_sol = sp.solvers.ode.dsolve( psi_region[ 0 ].last_step().lhs, 0, ivar = x )
#ics = { psi( 0 ): 0, psi( well_length ): 0 }

In [34]:
diff_sol

Eq(psi(x), C1*exp(-I*k_0*x) + C2*exp(I*k_0*x))

In [35]:
psi_region[ 0 ].add_step( diff_sol )

Eq(psi(x), C1*exp(-I*k_0*x) + C2*exp(I*k_0*x))

In [36]:
psi_region[ 0 ].rename( sp.Symbol( "C1" ), "A" )

Eq(psi(x), A*exp(-I*k_0*x) + C2*exp(I*k_0*x))

In [37]:
psi_region[ 0 ].rename( sp.Symbol( "C2" ), 0 )

Eq(psi(x), A*exp(-I*k_0*x))

In [38]:
psi_region[ 1 ].last_step()

Eq(V_0*psi(x) - hbar**2*Derivative(psi(x), (x, 2))/(2*m), E*psi(x))

In [39]:
psi_region[ 0 ].last_step()

Eq(psi(x), A*exp(-I*k_0*x))

In [40]:
psi_region[ 1 ].negate_add( Stepper.RIGHT )

Eq(-E*psi(x) + V_0*psi(x) - hbar**2*Derivative(psi(x), (x, 2))/(2*m), 0)

In [41]:
psi_region[ 1 ].manipulate( lambda side : side.collect( psi(x) ) )

Eq((-E + V_0)*psi(x) - hbar**2*Derivative(psi(x), (x, 2))/(2*m), 0)

In [42]:
psi_region[ 1 ].last_step().lhs.as_two_terms()[ 1 ]

-hbar**2*Derivative(psi(x), (x, 2))/(2*m)

In [43]:
region_1_clone = psi_region[ 1 ].clone()

region_1_clone.manipulate( lambda side : side 
                - region_1_clone.last_step()
                        .lhs.as_two_terms()[ 1 ]
        )

Eq((-E + V_0)*psi(x), hbar**2*Derivative(psi(x), (x, 2))/(2*m))

In [44]:
region_1_clone.manipulate( lambda side : side / region_1_clone.last_step().lhs.as_two_terms()[ 0 ] )

Eq(psi(x), hbar**2*Derivative(psi(x), (x, 2))/(2*m*(-E + V_0)))

In [45]:
region_1_clone.manipulate( lambda side : side / region_1_clone.last_step().rhs )

Eq(2*m*(-E + V_0)*psi(x)/(hbar**2*Derivative(psi(x), (x, 2))), 1)

In [46]:
region_1_clone.manipulate( lambda side : side * sp.Derivative( psi( x ), ( x, 2 ) ) )

Eq(2*m*(-E + V_0)*psi(x)/hbar**2, Derivative(psi(x), (x, 2)))

In [47]:
region_1_clone.manipulate( lambda side : side / psi( x ) )

Eq(2*m*(-E + V_0)/hbar**2, Derivative(psi(x), (x, 2))/psi(x))

In [48]:
k_1 = sp.Symbol( "k_1" )
k_1_squared = sp.Eq( k_1 ** 2, region_1_clone.last_step().lhs )

In [49]:
k_1_squared

Eq(k_1**2, 2*m*(-E + V_0)/hbar**2)

In [50]:
region_1_clone.manipulate( lambda side : side.subs( { k_1_squared.rhs : k_1_squared.lhs } ) )

Eq(k_1**2, Derivative(psi(x), (x, 2))/psi(x))

In [51]:
region_1_clone.manipulate( lambda side : side * psi( x ) )

Eq(k_1**2*psi(x), Derivative(psi(x), (x, 2)))

In [52]:
region_1_clone.negate_add( Stepper.RIGHT )

Eq(k_1**2*psi(x) - Derivative(psi(x), (x, 2)), 0)

In [53]:
psi_region[ 1 ] = region_1_clone

In [54]:
psi_region[ 1 ].last_step()

Eq(k_1**2*psi(x) - Derivative(psi(x), (x, 2)), 0)

In [55]:
psi_1 = sp.Function( "psi_1" )

In [56]:
psi_2 = sp.Function( "psi_2" )

In [57]:
psi_region[ 1 ].rename( psi( x ), psi_1( x ) )

Eq(k_1**2*psi_1(x) - Derivative(psi_1(x), (x, 2)), 0)

In [58]:
barrier_length = TunnelPotential.DEFAULT_WELL_LENGTH

In [59]:
sp.Derivative( psi_1( 0 ), x )

Derivative(psi_1(0), x)

In [60]:
#sp.constraint

In [61]:
diff_sol = sp.solvers.ode.dsolve( psi_region[ 1 ].last_step().lhs, 0, ivar = x, 
        ics = { 
                psi_1( 0 ) : psi( 0 ), 
                psi_1( barrier_length ) : psi_2( barrier_length ), 
                psi_1( x ).diff( x, 1 ).subs( x, 0 ) : psi( x ).diff( x, 1 ).subs( x, 0 ), 
                psi_1( x ).diff( x, 1 ).subs( x, barrier_length ) : psi_2( x ).diff( x, 1 ).subs( x, barrier_length )
        }
    )

ValueError: Couldn't solve for initial conditions

In [62]:
diff_sol_diff = sp.solvers.ode.dsolve( psi_region[ 1 ].last_step().lhs, 0, ivar = x, 
        ics = { 
                #psi_1( 0 ) : psi( 0 ), 
                #psi_1( barrier_length ) : psi_2( barrier_length ), 
                psi_1( x ).diff( x, 1 ).subs( x, 0 ) : psi( x ).diff( x, 1 ).subs( x, 0 ), 
                psi_1( x ).diff( x, 1 ).subs( x, barrier_length ) : psi_2( x ).diff( x, 1 ).subs( x, barrier_length )
        }
    )

In [63]:
diff_sol_plaine = sp.solvers.ode.dsolve( psi_region[ 1 ].last_step().lhs, 0, ivar = x, 
        ics = { 
                psi_1( 0 ) : psi( 0 ), 
                psi_1( barrier_length ) : psi_2( barrier_length ), 
                #psi_1( x ).diff( x, 1 ).subs( x, 0 ) : psi( x ).diff( x, 1 ).subs( x, 0 ), 
                #psi_1( x ).diff( x, 1 ).subs( x, barrier_length ) : psi_2( x ).diff( x, 1 ).subs( x, barrier_length )
        }
    )

In [66]:
diff_sol_diff

Eq(psi_1(x), (exp(L*k_1)*Derivative(psi_2(L), L)/(k_1*exp(2*L*k_1) - k_1) - Subs(Derivative(psi(x), x), x, 0)/(k_1*exp(2*L*k_1) - k_1))*exp(k_1*x) + (-exp(2*L*k_1)*Subs(Derivative(psi(x), x), x, 0)/(k_1*exp(2*L*k_1) - k_1) + exp(L*k_1)*Derivative(psi_2(L), L)/(k_1*exp(2*L*k_1) - k_1))*exp(-k_1*x))

In [65]:
diff_sol_plaine

Eq(psi_1(x), (-psi(0)/(exp(2*L*k_1) - 1) + psi_2(L)*exp(L*k_1)/(exp(2*L*k_1) - 1))*exp(k_1*x) + (psi(0)*exp(2*L*k_1)/(exp(2*L*k_1) - 1) - psi_2(L)*exp(L*k_1)/(exp(2*L*k_1) - 1))*exp(-k_1*x))

In [67]:
sp.Eq( diff_sol_diff.rhs, diff_sol_plaine.rhs ).simplify()

Eq(((psi(0)*exp(L*k_1) - psi_2(L))*exp(L*k_1) - (psi(0) - psi_2(L)*exp(L*k_1))*exp(2*k_1*x))*exp(-k_1*x)/(exp(2*L*k_1) - 1), ((exp(L*k_1)*Derivative(psi_2(L), L) - Subs(Derivative(psi(x), x), x, 0))*exp(2*k_1*x) - (exp(L*k_1)*Subs(Derivative(psi(x), x), x, 0) - Derivative(psi_2(L), L))*exp(L*k_1))*exp(-k_1*x)/(k_1*(exp(2*L*k_1) - 1)))